## **Análisis Calidad Gasolina - Repúlica Dominicana**

**Fecha:** Julio 2024

**Preparado por:** Rafael J. Mateo C.

**Contacto:** rmateo@squalitas.com

In [1]:
from pypdf import PdfReader, PdfWriter
from pypdf.errors import PdfStreamError
from ocrmypdf.exceptions import SubprocessOutputError
import os
import ocrmypdf
from tqdm import tqdm
from IPython.display import clear_output
import json
import pandas as pd
from typing import Callable, Generator
from datetime import datetime
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import HttpResponseError
from azure.ai.formrecognizer import DocumentAnalysisClient
from io import BufferedReader
from dotenv import load_dotenv
from pathlib import Path
from dateutil.parser import parse, ParserError
import re
import traceback
from collections import defaultdict
from unidecode import unidecode
import numpy as np

In [3]:
skip_files = []
files_with_error = []
processed_files = []

raw_dir = './files/raw'
tmp_dir = './files/tmp'
output_dir = './files/ocr'
json_dir = './files/json'
sample_dir = './files/sample'
log_dir = './var'

In [14]:
for dirname in [raw_dir, tmp_dir, output_dir, json_dir, log_dir]:
    if not os.path.exists(dirname):
        os.makedirs(dirname)

### **1. Preprocesamiento** 

#### **1.1. Eliminando Páginas que no contienen los datos numéricos de las pruebas**

El siguiente código elimina los arhivos que tengan cuatro páginas o menos, ya que estos son los reportes de las estaciones que no autorizaron la realización de los ensayos. También elimina las últimas dos páginas, las cuales solo contienen las imágenes de los ensayos y las firmas de los que realizaron el reporte.

In [46]:
for name in os.listdir(raw_dir):
    src = os.path.join(raw_dir, name)
    reader = PdfReader(src)
        
    if len(reader.pages) <= 4:
        skip_files.append(name)
        continue
    
    dest = os.path.join(tmp_dir, name)
    
    #Eliminar las últimas dos páginas, ya que son fotos y firmas
    for page_num in [reader.get_num_pages()-1,reader.get_num_pages()-2,0]:
        reader.remove_page(page_num)
    
    writer = PdfWriter(clone_from=reader)
    writer.write(dest)

/Prev=0 in the trailer - assuming there is no previous xref table
/Prev=0 in the trailer - assuming there is no previous xref table
/Prev=0 in the trailer - assuming there is no previous xref table
/Prev=0 in the trailer - assuming there is no previous xref table
/Prev=0 in the trailer - assuming there is no previous xref table
/Prev=0 in the trailer - assuming there is no previous xref table
/Prev=0 in the trailer - assuming there is no previous xref table
/Prev=0 in the trailer - assuming there is no previous xref table
/Prev=0 in the trailer - assuming there is no previous xref table
/Prev=0 in the trailer - assuming there is no previous xref table
/Prev=0 in the trailer - assuming there is no previous xref table
/Prev=0 in the trailer - assuming there is no previous xref table
/Prev=0 in the trailer - assuming there is no previous xref table
/Prev=0 in the trailer - assuming there is no previous xref table
/Prev=0 in the trailer - assuming there is no previous xref table
/Prev=0 in

Un total de 130 archivos contienen cuatro páginas o menos, por lo que fueron omitidos para este análisis.

In [42]:
len(skip_files)

130

#### **1.2. Realizando OCR**

En la siguiente celda realizaremos un OCR a aquellos archivos que contienen más de 4 páginas. Los archivos PDFs con OCR son colocados en el directorio 'files/ocr' por si requieren ser consultados posteriormente.

In [77]:
import warnings

error_files = []
warning_files = []

for name in tqdm(os.listdir(tmp_dir), unit='file', desc='Files processed: '):
    
    #Omite los archivos que tienen cuatro páginas o menos
    if name in skip_files or os.path.exists(os.path.join(output_dir, name)):
        clear_output(wait=True)
        continue
    
    try:
        with warnings.catch_warnings(record = True) as w:
            
            warnings.simplefilter("always")
            #Realiza OCR
            ocrmypdf.ocr(os.path.join(tmp_dir, name),
                    os.path.join(output_dir, name),skip_text=True, language='spa', deskew=True)
            if w:
                raise w[-1].category(w[-1].message)
    
    except SubprocessOutputError as error:
        error_files.append({'file': name, 'error': error})
    
    except Warning as e:
        warning_files.append({'file': name, 'warning': e})
        
    clear_output(wait=True)

Files processed: 100%|██████████| 810/810 [1:33:18<00:00,  6.91s/file]


Solo un archivo procesado generó un error. Este se refiere al reporte "2023 03 047.pdf". Revisando el archivo manualmente, la razón del error se debe a que las primeras páginas del reporte ya están en texto y no escaneado. Como es solo un archivo, manualmente pueden eliminarse las primeras cuantro páginas para realizar el OCR a las tablas de los datos solamente, ya que estas si están escaneadas.

In [78]:
error_files

[{'file': '2023 03 047.pdf',
  'error': ocrmypdf.exceptions.SubprocessOutputError('Ghostscript PDF/A rendering failed')}]

#### **1.3. Eliminando Páginas**

Ahora procedemos a eliminar las páginas que no contengan tablas de datos. Estas páginas suelen contener portadas, resúmenes u otras informaciones no relevantes para este análisis. La forma de filtrar las páginas que nos interesan es identificando aquellas que tengan el texto "Test Report".

In [50]:

def remove_pages(compare_func: Callable, input_dir: str, output_dir: str) ->dict[str, dict|list]:

    results = {'deleted_pages':defaultdict(list), 'errors': {} }

    for name in tqdm(os.listdir(input_dir), unit='file', desc='Files processed: '):
        
        try:
            if not name.endswith('.pdf'):
                continue
            
            reader = PdfReader(os.path.join(input_dir, name))
            pages = list(reader.pages)
        
        except PdfStreamError as e:
            results['errors'][name] = e
            continue
        
        for i,page in enumerate(pages):   
            text = page.extract_text()
            
            if not compare_func(text):
    
                reader.remove_page(page, True)
                results['deleted_pages'][name].append(i+1)
                
        writer = PdfWriter(clone_from=reader)
        writer.write(os.path.join(output_dir, name))
    
    return results

In [26]:
def page_count(dirname: str) -> int:
    num_pages = 0
    for name in tqdm(os.listdir(dirname), unit='file', desc='Files processed: '):
    
        try:
            if not name.endswith('.pdf'):
                continue
            
            reader = PdfReader(os.path.join(dirname, name))
            num_pages += len(reader.pages)

        except PdfStreamError as e:
            continue
    return num_pages

In [27]:
results = remove_pages(lambda text: text.find("Test Report") >= 0, output_dir, tmp_dir)

Files processed: 100%|██████████| 810/810 [03:08<00:00,  4.30file/s]


A continuación se muestran las páginas eliminadas para cada archivo

In [28]:
results

{'deleted_pages': defaultdict(list,
             {'2023 03 140.pdf': [1, 2, 3, 4],
              '2023 03 034.pdf': [2],
              '2023 05 065.pdf': [1, 2, 3, 4],
              '2023 05 059.pdf': [1],
              '2023 02 030.pdf': [2],
              '2023 08 047.pdf': [1],
              '2023 03 021.pdf': [4],
              '2023 07 001.pdf': [1, 2, 3],
              '2023 05 112.pdf': [3, 4],
              '2023 03 037.pdf': [3],
              '2023 05 072.pdf': [1],
              '2023 08 051.pdf': [1, 2],
              '2023 04 088.pdf': [1, 3],
              '2023 04 089.pdf': [1, 2, 3, 4],
              '2023 03 036.pdf': [4],
              '2023 03 022.pdf': [3],
              '2023 05 107.pdf': [1, 2, 3, 4],
              '2023 06 007.pdf': [1],
              '2023 01 080.pdf': [3],
              '2023 06 003.pdf': [2],
              '2023 06 017.pdf': [1, 2, 3, 4],
              '2023 04 066.pdf': [1],
              '2023 05 076.pdf': [1],
              '2023 06 016.pdf

Debido a que usaremos un servicio de pago de Azure, limitaremos el análisis únicamente a las Gasolinas. Para esto, filtramos solamente las páginas que contienen la palabra "GASOLINA". En caso de requerirse los datos para el Gasoil, simplemente sería sustituir la palabra "GASOLINA" por "GASOIL".

In [44]:
results2 = remove_pages(lambda text: text.find("GASOLINA") >= 0, tmp_dir,'./files/sample')

Files processed: 100%|██████████| 810/810 [03:09<00:00,  4.28file/s]


A continuación se presentan las páginas que fueron eliminadas para cada archivo

In [48]:
results2

{'deleted_pages': defaultdict(list,
             {'2023 07 014.pdf': [4, 5],
              '2023 07 028.pdf': [4, 5],
              '2023 04 100.pdf': [4, 5],
              '2023 01 051.pdf': [3, 4],
              '2023 01 045.pdf': [3, 4],
              '2023 01 079.pdf': [2, 5],
              '2023 06 011.pdf': [4, 5],
              '2023 06 005.pdf': [4, 5],
              '2023 06 039.pdf': [4, 5],
              '2023 03 168.pdf': [4, 5],
              '2023 05 111.pdf': [4, 5],
              '2023 05 105.pdf': [4, 5],
              '2023 03 034.pdf': [1, 3, 4],
              '2023 03 008.pdf': [4],
              '2023 08 052.pdf': [4],
              '2023 05 065.pdf': [2],
              '2023 05 071.pdf': [4, 5],
              '2023 08 046.pdf': [4, 5],
              '2023 05 059.pdf': [3],
              '2023 02 025.pdf': [4, 5],
              '2023 02 031.pdf': [3, 4, 5],
              '2023 07 148.pdf': [4, 5],
              '2023 02 019.pdf': [4, 5],
              '2023 04 074.

Finalmente, contamos la cantidad de páginas de nuestra muestra que serán analizadas en Azure, las cuales totalizan 2,284

In [51]:
page_count('./files/sample')

Files processed: 100%|██████████| 810/810 [00:00<00:00, 1109.38file/s]


2284

### **2. Análisis de los Documentos**

#### **2.1. Análisis y Extracción**

En esta sección se estará enviando cada uno de los documentos al servicio de Azure y luego se procederá a extraer la información de estos.

In [203]:
class DataExtractor:
    """
    Esta clase tiene como propósito extraer los datos del 
    json devuelto por el servicio de Azure.
    """
    
    page_num = None
    key_value_pairs: list[dict] = []
    tables: list[dict] = []
    
    def __init__(self, data) -> None:
        self.key_value_pairs = data['key_value_pairs']
        self.tables = data['tables']
            
    def __filter_by_content_fn (self,page_num: int, keyword: str) -> Callable:
        
        return lambda item: re.search(keyword, item['key']['content'].lower()) is not None \
            and item['key']['bounding_regions'][0]['page_number'] == page_num

    
    def __filter_table(self, page_number: int) -> dict|None:
        results = list(filter(
            lambda table: table['bounding_regions'][0]['page_number'] == page_number, 
            self.tables))
        
        if len(results) == 0:
            return None
        
        return results[0]
    
    def __extract_headers(self, cells: list[dict]) -> list[str]:
        filtered_cells =  list(filter(lambda item: 'kind' in item and item['kind'] == 'columnHeader', cells ))
        headers = []
        
        for header in filtered_cells:
            # No nos interesan las columnas combinadas
            if 'column_span' in header and header['column_span'] >= 2:
                continue
            
            headers.insert(header['column_index'], header['content'])
  
        return headers
    
    def __extract_rows(self, cells: list[dict], columnCount: int) -> list[str|float|int]:
        filtered_cells = list(filter(lambda item: 'kind' in item and item['kind'] == 'content', cells))
        rows = []
        row = []
        
        for index,cell in enumerate(filtered_cells):
            
            if index >0  and (index % columnCount) == 0:
                
                rows.append(row)
                row = []
                
            row.insert(cell['column_index'], cell['content'])
            
        return rows
    
    def extract_client_name (self, page_number: int) -> str:
        
        results = list(filter(self.__filter_by_content_fn(page_number, r"cliente[\s|\/\\n]+client"), self.key_value_pairs))
        
        if len(results) == 0:
            return ''
        
        return results[0]['value']['content']
        
    def extract_test_date (self, page_number: int) -> datetime|str:
        results = list(filter(
            self.__filter_by_content_fn(page_number, r"date-time"), self.key_value_pairs))
        
        if len(results) == 0:
            return ''
        date = re.match(
            r"(\d{1,2}/\d{1,2}/\d{4}\s\d{1,2}:\d{2}(?:\s[ap]\.m)?)", 
            results[0]['value']['content'])[0]
        try:
            return parse(date)
        except ParserError:
            return parse(re.sub(r'p\.m|a\.m', '',date))
        
    def extract_product_name(self, page_number: int) -> str:
            
        results = list(filter(self.__filter_by_content_fn(page_number, 'producto'), self.key_value_pairs))
        
        if len(results) == 0:
            return ''
        
        return results[0]['value']['content']
    
    def extract_table(self, page_num:int) -> pd.DataFrame:
        table_items =  self.__filter_table(page_num)
        if table_items is None:
            return pd.DataFrame
        
        headers = self.__extract_headers(table_items['cells'])
        values = self.__extract_rows(
            table_items['cells'], len(headers))

        df = pd.DataFrame(values, columns=headers)
        
        return df.assign(
            CLIENTE = self.extract_client_name(page_num), 
            PRODUCTO = self.extract_product_name(page_num), 
            FECHA = self.extract_test_date(page_num))
    
    def extract_tables(self) -> pd.DataFrame:
        df = pd.DataFrame()
        
        for table in self.tables:
            df1 = self.extract_table(table['bounding_regions'][0]['page_number'])
            df1['NUM PAGINA'] = table['bounding_regions'][0]['page_number']
            df = pd.concat([df, df1], ignore_index=True)
            
        return df
    

In [131]:
class DocumentAnalyzer:
    """
    Esta clase se encarga de leer cada uno de los archivos
    en el directorio especificado y enviarlos al servicio de
    Azure
    """
    
    document_analysis_client: DocumentAnalysisClient|None = None
    processed_files: list[str] = []
    errors: list[dict[str,str|BaseException]] = []
    
    def __init__(self) -> None:
        
        load_dotenv(override=True)
        endpoint = os.environ["AZURE_FORM_RECOGNIZER_ENDPOINT"]
        key = os.environ["AZURE_FORM_RECOGNIZER_KEY"]
        
        self.document_analysis_client = DocumentAnalysisClient(
        endpoint=endpoint, credential=AzureKeyCredential(key)
    )
        
    def __file_reader(self,dirname: str) -> Generator[BufferedReader, None, None]:
        
        for name in tqdm(os.listdir(dirname), unit='file', desc='Files processed: '):
            with open(os.path.join(dirname, name), "rb") as f:
                yield f
    
    def __get_analysis(self, file: BufferedReader) -> dict[str, str]:
        
        poller = self.document_analysis_client.begin_analyze_document(
                    "prebuilt-document", document=file
                )
        return poller.result().to_dict()
    
    def __write_results(self,filename: str, results: dict[str, str], output_dir: str, log_dir = './var') -> None:
        
        with open (os.path.join(output_dir, filename + '.json'), 'w') as json_file, \
            open(os.path.join(log_dir, 'checkpoint.txt'), 'w') as log_file:
                    
            json_file.write(json.dumps(results))
            log_file.write(json.dumps(processed_files))
        
    def analyze(self,input_dir: str, output_dir: str, log_dir = './var') -> pd.DataFrame:
        
        df = pd.DataFrame()
        self.processed_files = []
        self.errors = []
    
        for file in self.__file_reader(input_dir):
            
            try:
                filename = Path(output_dir, file.name).stem
                
                result =  self.__get_analysis(file)
                
                extractor = DataExtractor(result)
                extracted_df = extractor.extract_tables().assign(ARCHIVO=filename + '.pdf')
                
                df = pd.concat([df,extracted_df], ignore_index = True)
                    
                self.processed_files.append(file.name)
    
            except (HttpResponseError, Exception) as e:
                error = {
                    'file': file.name, 
                    'error': str(e), 
                    'traceback': traceback.extract_tb(e.__traceback__).format()}
            
                if e is HttpResponseError:
                    error['comment'] = 'Archivo no procesado'
                    result = None 
                
                self.errors.append(error)      
                
            finally:
                if result is not None:
                    self.__write_results(filename, result, output_dir, log_dir)
        return df

Procedemos a realizar el análisis de los documentos y extraer la información. En total se procesarán 809 archivos.

In [132]:
analyzer = DocumentAnalyzer()
df = analyzer.analyze(sample_dir, json_dir)

Files processed: 100%|██████████| 809/809 [3:35:36<00:00, 15.99s/file]    


Hacemos una copia del dataframe original para fines de respaldo.

In [136]:
df_copy = df.copy()

En total se tiene más de 31,000 filas de información

In [134]:
len(df)

31470

Como se puede ver a continuación, algunas columnas están duplicadas, o bien, parte del texto del archivo fue leído como una columna.

In [143]:
df.columns

Index(['ANÁLISIS / ANALYSES', 'UNIDADES / UNIT', 'MIN', 'MAX',
       'MÉTODO / METHOD', 'RESULTADOS DE CALIDAD / QUALITY RESULTS', 'CLIENTE',
       'PRODUCTO', 'FECHA', 'NUM PAGINA', 'ARCHIVO', '',
       'INCERTIDUMBRE/ UNCERTAINTY %', 'MEIN', 'млх', 'METODO / METHOD',
       '.MAX', '-MAX', 'DICERTIDUMBRE7. UNCERTAINTY %.', 'MEN', ': MAX',
       'BIIN', 'PRODUCTO / PRODUCT', 'ISLA #2', ': GASOLINA REGULAR', 'N/A',
       '. MAX', 'MĒTODO / METHOD', 'METODO / METIIOD', '.. ISLA #2',
       'Producto(s) / Product(s)', 'GASOLINA REGULAR GASOIL REGULAR',
       'VESSEL NAME (Nombre del Buque)', 'NAVIG8 STRENGTH',
       'VESSEL REPORT / (REPORTE\nPRODUCT', 'DEL BUQUE)\n(PRODUCTO)',
       'GASOLINA REGULAR', 'GASOIL REGULAR',
       'OVERALL ANALYSIS (ANÁLISIS TOTAL)',
       'PUERTO DE CARGA) (CIFRAS EN EL LOAD PORT B/L',
       '(CIFRAS EN EL PUERTO DE DESCARGA) VESSEL',
       '(GANANCIA / PERDIDA) GAIN/LOSS', ': N/A', 'BIEN',
       ': GASOLINA RACIAL.AR', 'MÁX', 'UNCERTAINTY %', 

Para limpiar el dataframe, se define un mapa con palabras claves de las columnas repetidas e indicando el nombre correcto de la columna.

In [144]:
column_map = {
    'UNIDADES': 'UNIDADES / UNIT',
    'UNIT': 'UNIDADES / UNIT',
    'CERTIDUMBRE': 'INCERTIDUMBRE/ UNCERTAINTY %',
    'RESULTADOS': 'RESULTADOS DE CALIDAD / QUALITY RESULTS',
    'MÉTODO': 'MÉTODO / METHOD',
    'METODO': 'MÉTODO / METHOD',
    'MI': 'MIN',
    'MA': 'MAX',
    'ANALYSES': 'ANÁLISIS / ANALYSES'
}

Usando el método "combine_first" de pandas, movemos los datos de las columnas duplicadas a la columna que le corresponde. Posteriormente se procede a eliminar la columna duplicada.

In [145]:
for keyword,column_name in column_map.items():
    
    try:
        target_cols = list(filter(lambda col: col.find(keyword) >= 0 or \
            unidecode(col).find(keyword) >= 0, df.columns.to_list()))
        
        target_cols.remove(column_name)
        
        for col in target_cols:
            df[column_name].combine_first(df[col])
            df.drop(col, axis = 1, inplace=True)
    except ValueError as e:
        print(f'{column_name} not in list')

De las 176 columnas que existían, ahora quedan cerca de 40. Como se observa, las columnas 11 y de la 13 en adelante pueden eliminarse, pues la mayoría de las filas de estas columnas contienen valores nulos.

Por otro lado, la columna 'Fecha' está siendo detectada como tipo objeto, cuando debería ser datetime.

In [147]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31470 entries, 0 to 31469
Data columns (total 40 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   ANÁLISIS / ANALYSES                           30799 non-null  object
 1   UNIDADES / UNIT                               29354 non-null  object
 2   MIN                                           30832 non-null  object
 3   MAX                                           31079 non-null  object
 4   MÉTODO / METHOD                               30023 non-null  object
 5   RESULTADOS DE CALIDAD / QUALITY RESULTS       30255 non-null  object
 6   CLIENTE                                       31470 non-null  object
 7   PRODUCTO                                      31470 non-null  object
 8   FECHA                                         31470 non-null  object
 9   NUM PAGINA                                    31470 non-null  int64 
 10

Convertimos la columna fecha al tipo datetime

In [196]:
df['FECHA'] = pd.to_datetime(df['FECHA'])
df['ANÁLISIS / ANALYSES'] = df['ANÁLISIS / ANALYSES'].astype(str)

Eliminamos las columnas 11 y 13 en adelante

In [167]:
df.drop(df.columns[np.r_[11, 13:len(df.columns)]], axis=1, inplace=True)

A continuación, se presenta la tabla final.

In [168]:
df.head(10)

,ANÁLISIS / ANALYSES,UNIDADES / UNIT,MIN,MAX,MÉTODO / METHOD,RESULTADOS DE CALIDAD / QUALITY RESULTS,CLIENTE,PRODUCTO,FECHA,NUM PAGINA,ARCHIVO,INCERTIDUMBRE/ UNCERTAINTY %
0,NUMERO DE OCTANO METODO RESEARCH (RON),-,95,-,ASTM D-2699,96.0,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,NaN
1,NUMERO DE OCTANO METODO MOTOR (MON),.,82,-,ASTM D-2700,87.4,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,NaN
2,CONTENIDO DE PLOMO,G/gal,1,0.02,ASTM D-3237,N/D,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,NaN
3,PRESION A VAPOR REID (RVP) @ 100 º F,PSI,1,10.0,ASTM D-323,8.02,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,NaN
4,RVP + 0.1 E 70 ° C,1,Reportar,,Calculo,9.74,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,NaN
5,DESTILACION 10% VOL. RECUPERADO,0,",",75,ASTM D86,62,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,NaN
6,DESTILACION 50% VOL. RECUPERADO,·℃,,121,ASTM D86,110,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,NaN
7,DESTILACION 90% VOL. RECUPERADO,°℃,1,190,ASTM D86,159,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,NaN
8,PUNTO FINAL,℃,,225,ASTM D86,202,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,NaN
9,RESIDUO,% Vol.,,"2,0",ASTM D86,0.5,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,NaN


Guardamos una copia de la tabla en formato parquet y csv.

In [198]:
df.to_parquet('./gas_station_dataset.parquet')

In [199]:
df.to_csv('./gas_station_dataset.csv')

#### **2.2. Análisis de los Errores**

Un total de 99 archivos no pudieron ser procesados. Se procederá a revisar el log de los errores encontrados.

In [170]:
len(analyzer.errors)

99

In [201]:
errors = []

for item in analyzer.errors:
    errors.append({
        'file': item['file'], 
        'error': str(item['error']), 
        'traceback': traceback.extract_tb(item['error'].__traceback__).format()})



Revisando los errores, al parecer algunos datos de la columna fecha no fueron extraidos correctamente. Es posible que esto se deba a un problema en el formato de fecha como se encuentra en el reporte.

In [206]:
print(json.dumps(errors[:5], indent=4))

[
    {
        "file": "./files/sample/2023 01 079.pdf",
        "error": "'NoneType' object is not subscriptable",
        "traceback": [
            "  File \"/var/folders/gv/t0xf0frn51g8g_xd29ccdwzc0000gn/T/ipykernel_58275/3819070703.py\", line 58, in analyze\n    extracted_df = extractor.extract_tables().assign(ARCHIVO=filename + '.pdf')\n                   ^^^^^^^^^^^^^^^^^^^^^^^^^^\n",
            "  File \"/var/folders/gv/t0xf0frn51g8g_xd29ccdwzc0000gn/T/ipykernel_58275/1914603399.py\", line 108, in extract_tables\n    df1 = self.extract_table(table['bounding_regions'][0]['page_number'])\n          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n",
            "  File \"/var/folders/gv/t0xf0frn51g8g_xd29ccdwzc0000gn/T/ipykernel_58275/1914603399.py\", line 102, in extract_table\n    FECHA = self.extract_test_date(page_num))\n            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n",
            "  File \"/var/folders/gv/t0xf0frn51g8g_xd29ccdwzc0000gn/T/ipykernel_58275/191

Procedemos a guardar el reporte de los errores en un archivo json para poder analizarlos más adelante.

In [202]:
with open('./var/errors_log.json', 'w') as f:
   json.dump(errors, f,indent=4, sort_keys=True)

#### **2.3. Verificación de Calidad**

En esta sección tomaremos una muestra aleatoria para revisar manualmente los datos extraidos con los archivos originales, y así asegurarnos que los datos sean correctos. Para determinar el tamaño de muestra, así como los criterios de aceptación o rechazo, se procedió a usar la tabla ANSI Z1.4 con un nivel de inspección general I, un tamaño de lote de 700 archivos y un AQL de 0.40.

Por limitaciones de recursos, la verificación de la muestra contra los archivos originales se limitará a los valores RON y MON solamente.

In [282]:
sample = df['ARCHIVO'].sample(n = 32, random_state=42)

In [283]:
sample_df = df[df['ARCHIVO'].isin(sample)][df['ANÁLISIS / ANALYSES'].str.contains('RON') | df['ANÁLISIS / ANALYSES'].str.contains('MON')]

/var/folders/gv/t0xf0frn51g8g_xd29ccdwzc0000gn/T/ipykernel_58275/831347874.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sample_df = df[df['ARCHIVO'].isin(sample)][df['ANÁLISIS / ANALYSES'].str.contains('RON') | df['ANÁLISIS / ANALYSES'].str.contains('MON')]


**Resultados de la verificación: PENDIENTE**

In [290]:
sample_df[20:54]

,ANÁLISIS / ANALYSES,UNIDADES / UNIT,MIN,MAX,MÉTODO / METHOD,RESULTADOS DE CALIDAD / QUALITY RESULTS,CLIENTE,PRODUCTO,FECHA,NUM PAGINA,ARCHIVO,INCERTIDUMBRE/ UNCERTAINTY %
6620,NUMERO DE OCTANO METODO RESEARCH (RON),,89,.,ASTM D-2699,92.8,NEXT LICEY,GASOLINA REGULAR,2023-03-27 18:50:00,2,2023 03 144.pdf,NaN
6621,NUMERO DE OCTANO METODO MOTOR (MON),.,76,,ASTM D-2700,81.9,NEXT LICEY,GASOLINA REGULAR,2023-03-27 18:50:00,2,2023 03 144.pdf,NaN
6642,NUMERO DE OCTANO METODO RESEARCH (RON),,89,,ASTM D-2699,93.2,NEXT LICEY,GASOLINA REGULAR,2023-03-27 18:50:00,3,2023 03 144.pdf,NaN
6643,NUMERO DE OCTANO METODO MOTOR (MON),,76,-,ASTM D-2700,84.2,NEXT LICEY,GASOLINA REGULAR,2023-03-27 18:50:00,3,2023 03 144.pdf,NaN
7793,NUMERO DE OCTANO METODO RESEARCH (RON),,95,,ASTM D-2699,93.6,TOTAL VISTA DE JIMENOA\nDÍA ANÁLISIS,GASOLINA PREMIUM,2023-03-04 07:28:00,1,2023 04 003.pdf,NaN
7794,NUMERO DE OCTANO METODO MOTOR (MON),,82,.,ASTM D-2700,84.9,TOTAL VISTA DE JIMENOA\nDÍA ANÁLISIS,GASOLINA PREMIUM,2023-03-04 07:28:00,1,2023 04 003.pdf,NaN
7815,NUMERO DE OCTANO METODO RESEARCH (RON),,89,.,ASTM D-2699,91.5,TOTAL VISTA DE JIMENOA,GASOLINA REGULAR,2023-03-04 07:28:00,2,2023 04 003.pdf,NaN
7816,NUMERO DE OCTANO METODO MOTOR (MON),1,76,,ASTM D-2700,81.2,TOTAL VISTA DE JIMENOA,GASOLINA REGULAR,2023-03-04 07:28:00,2,2023 04 003.pdf,NaN
7837,NUMERO DE OCTANO METODO RESEARCH (RON),,89,.,ASTM D-2699,91.5,TOTAL VISTA DE JIMENOA,GASOLINA REGULAR,2023-03-04 07:28:00,3,2023 04 003.pdf,NaN
7838,NUMERO DE OCTANO METODO MOTOR (MON),-,76,-,ASTM D-2700,81.4,TOTAL VISTA DE JIMENOA,GASOLINA REGULAR,2023-03-04 07:28:00,3,2023 04 003.pdf,NaN


3. Extracción de Otros Datos

In [202]:
df = pd.read_parquet('./gas_station_dataset.parquet')

In [203]:
df_address = pd.read_json('./address.json')

In [204]:
df_address.head()

,file,address
0,2023 07 014.pdf,"AIVEPET, S.R.L— REP. DOMINICANA\nReporte Fo..."
1,2023 07 028.pdf,"AVEPET, S.RL—REP.DOMINICANA\nReporte Fotográ..."
2,2023 04 100.pdf,"AINVEPET, S.RL — REP. DOMINICANA\nReporte F..."
3,2023 01 051.pdf,"AIVEPET, S.R.L — REP. DOMINICANA\nReporte F..."
4,2023 01 045.pdf,"AIVEPET, S.R.L — REP. DOMINICANA\nReporte Fo..."


In [205]:
df = df.merge(df_address, left_on='ARCHIVO', right_on='file')

In [206]:
df.drop('file', axis=1, inplace=True)

In [207]:
df = df.rename(columns = {
    'ANÁLISIS / ANALYSES': 'ANÁLISIS', 
    'MÉTODO / METHOD': 'MÉTODO', 
    'RESULTADOS DE CALIDAD / QUALITY RESULTS': 'RESULTADOS',
    'CLIENTE': 'NOMBRE ESTACIÓN',
    'address': 'DIRECCIÓN', 'UNIDADES / UNIT': 'UNIDADES'
    })

In [208]:
df.head()

,ANÁLISIS,UNIDADES,MIN,MAX,MÉTODO,RESULTADOS,NOMBRE ESTACIÓN,PRODUCTO,FECHA,NUM PAGINA,ARCHIVO,INCERTIDUMBRE/ UNCERTAINTY %,DIRECCIÓN
0,NUMERO DE OCTANO METODO RESEARCH (RON),-,95,-,ASTM D-2699,96.0,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,None,"AIVEPET, S.R.L— REP. DOMINICANA\nReporte Fo..."
1,NUMERO DE OCTANO METODO MOTOR (MON),.,82,-,ASTM D-2700,87.4,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,None,"AIVEPET, S.R.L— REP. DOMINICANA\nReporte Fo..."
2,CONTENIDO DE PLOMO,G/gal,1,0.02,ASTM D-3237,N/D,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,None,"AIVEPET, S.R.L— REP. DOMINICANA\nReporte Fo..."
3,PRESION A VAPOR REID (RVP) @ 100 º F,PSI,1,10.0,ASTM D-323,8.02,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,None,"AIVEPET, S.R.L— REP. DOMINICANA\nReporte Fo..."
4,RVP + 0.1 E 70 ° C,1,Reportar,,Calculo,9.74,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,None,"AIVEPET, S.R.L— REP. DOMINICANA\nReporte Fo..."


In [209]:
df['PRODUCTO'].unique()

array(['GASOLINA PREMIUM', 'GASOLINA REGULAR', ': GASOLINA REGULAR',
       'GASOLINA. PREMIUM', 'GASOLINA PREMIUM.', 'GASOLINA PREMIJM',
       'GASOLINA PREMIIM', 'GASOLINA REGULAR GASOIL REGULAR', '',
       'GASOLINA PREMIUM\nISLA # 2', 'GASOLINA PREMKIM',
       'GASOLINA PREMIUM\nISLA', 'GASOLINA RACIAL.AR',
       ': GASOLINA PREMIUM', 'GASOLINA PREMIOM'], dtype=object)

In [210]:
df['PRODUCTO'] = df['PRODUCTO'].apply(lambda item: 'GASOLINA PREMIUM' if ('PREM' in item)  else item)

In [211]:
df['PRODUCTO'] = df['PRODUCTO'].apply(lambda item: 'GASOLINA REGULAR' if ('REG' in item)  else item)

In [212]:
df = df[df['PRODUCTO'] != '']

In [213]:
df['PRODUCTO'].unique()

array(['GASOLINA PREMIUM', 'GASOLINA REGULAR', 'GASOLINA RACIAL.AR'],
      dtype=object)

In [214]:
df['PRODUCTO'] = df['PRODUCTO'].apply(lambda item: 'GASOLINA REGULAR' if (item == 'GASOLINA RACIAL.AR')  else item) 

In [215]:
df['PRODUCTO'].unique()

array(['GASOLINA PREMIUM', 'GASOLINA REGULAR'], dtype=object)

In [216]:
def get_gas_station_name(row: pd.Series) -> str:
    
    stations = [
        'ATLANTIC', 'TEXACO', 'SHELL', 'ECOPETROLEO', 'ESSO', 'NEXT',
       'SUNIX', 'PETROMOVIL', 'PETRONAN', 'TOTAL', 'AXXON', 'TDC', 'UNITED',
       'SOL', 'NATIVA', 'SIGMA', 'SITRAS', 'EXCOM'
    ]
   # print(row['CLIENTE'].contains('A'))
    for station in stations:
        if station in row['NOMBRE ESTACIÓN']:
            break
        
    return station

In [217]:
df['MARCA ESTACION'] = None
df['MARCA ESTACION'] = df.apply(lambda row: get_gas_station_name(row), axis=1)

In [218]:
df.head()

,ANÁLISIS,UNIDADES,MIN,MAX,MÉTODO,RESULTADOS,NOMBRE ESTACIÓN,PRODUCTO,FECHA,NUM PAGINA,ARCHIVO,INCERTIDUMBRE/ UNCERTAINTY %,DIRECCIÓN,MARCA ESTACION
0,NUMERO DE OCTANO METODO RESEARCH (RON),-,95,-,ASTM D-2699,96.0,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,None,"AIVEPET, S.R.L— REP. DOMINICANA\nReporte Fo...",ATLANTIC
1,NUMERO DE OCTANO METODO MOTOR (MON),.,82,-,ASTM D-2700,87.4,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,None,"AIVEPET, S.R.L— REP. DOMINICANA\nReporte Fo...",ATLANTIC
2,CONTENIDO DE PLOMO,G/gal,1,0.02,ASTM D-3237,N/D,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,None,"AIVEPET, S.R.L— REP. DOMINICANA\nReporte Fo...",ATLANTIC
3,PRESION A VAPOR REID (RVP) @ 100 º F,PSI,1,10.0,ASTM D-323,8.02,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,None,"AIVEPET, S.R.L— REP. DOMINICANA\nReporte Fo...",ATLANTIC
4,RVP + 0.1 E 70 ° C,1,Reportar,,Calculo,9.74,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,None,"AIVEPET, S.R.L— REP. DOMINICANA\nReporte Fo...",ATLANTIC


In [220]:
results = []
no_matches = []
for name in os.listdir('./files/json'):
    
    if not name.endswith('.json'):
        continue
    
    with open(os.path.join('./files/json', name), 'r') as f:
        json_data = json.load(f)
        obs_items = list(filter(lambda item: re.match(r'(observ.*?|valor.*?|signifi.*?|encuent.*?)', item['content'],re.DOTALL | re.IGNORECASE) is not None, json_data['paragraphs']))
    
    
    if len(obs_items) == 0:
        no_matches.append({'file': name})
        continue
    
    for item in obs_items:
        results.append ({'file': name, 'text': item['content'], 'page': item['bounding_regions'][0]['page_number']})
        

In [229]:
df_obs = pd.DataFrame(results)

In [237]:
len(df_obs)

3042

In [231]:
df_obs_copy = df_obs.copy()

In [232]:
duplicated = df_obs[df_obs.duplicated(subset=['file', 'page'], keep='first')]
df_obs_no_duplicates = df_obs.drop_duplicates(subset=['file', 'page'], keep='first')

In [233]:
for index,row in duplicated.iterrows():
    df_obs_no_duplicates.loc[(df_obs_no_duplicates['file'] == row['file']) & (df_obs_no_duplicates['page'] == row['page']), 'text'] += ' '+ row['text']

In [234]:
df_obs_no_duplicates

,file,text,page
0,2023 08 030.json,Observación: Valores resaltados se encuentran ...,1
3,2023 08 030.json,Observación: Valores resaltados se encuentran ...,2
4,2023 08 030.json,Observación: Valores reportados se encuentran ...,3
5,2023 01 023.json,Observación: Valores reportados se encuentran ...,1
6,2023 01 023.json,Observación: Valores resaltados se encuentran ...,2
...,...,...,...
3036,2023 04 079.json,Observación: Valores resaltados se encuentran ...,3
3037,2023 05 079.json,Observación: Valores resaltados se encuentran ...,1
3038,2023 05 079.json,Observación: Valores resaltados se encuentran ...,2
3039,2023 05 079.json,Observación: Valores reportados se encuentran ...,3


In [235]:
len(no_matches)

1

In [236]:
len(df_obs_no_duplicates[df_obs_no_duplicates['text'].str.contains('fuera')])/len(df_obs_no_duplicates)

0.6953528399311532

In [ ]:
df_copy = df.copy()

In [238]:
df.columns

Index(['ANÁLISIS', 'UNIDADES', 'MIN', 'MAX', 'MÉTODO', 'RESULTADOS',
       'NOMBRE ESTACIÓN', 'PRODUCTO', 'FECHA', 'NUM PAGINA', 'ARCHIVO',
       'INCERTIDUMBRE/ UNCERTAINTY %', 'DIRECCIÓN', 'MARCA ESTACION'],
      dtype='object')

In [241]:
df.head()

,ANÁLISIS,UNIDADES,MIN,MAX,MÉTODO,RESULTADOS,NOMBRE ESTACIÓN,PRODUCTO,FECHA,NUM PAGINA,ARCHIVO,INCERTIDUMBRE/ UNCERTAINTY %,DIRECCIÓN,MARCA ESTACION
0,NUMERO DE OCTANO METODO RESEARCH (RON),-,95,-,ASTM D-2699,96.0,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,None,"AIVEPET, S.R.L— REP. DOMINICANA\nReporte Fo...",ATLANTIC
1,NUMERO DE OCTANO METODO MOTOR (MON),.,82,-,ASTM D-2700,87.4,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,None,"AIVEPET, S.R.L— REP. DOMINICANA\nReporte Fo...",ATLANTIC
2,CONTENIDO DE PLOMO,G/gal,1,0.02,ASTM D-3237,N/D,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,None,"AIVEPET, S.R.L— REP. DOMINICANA\nReporte Fo...",ATLANTIC
3,PRESION A VAPOR REID (RVP) @ 100 º F,PSI,1,10.0,ASTM D-323,8.02,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,None,"AIVEPET, S.R.L— REP. DOMINICANA\nReporte Fo...",ATLANTIC
4,RVP + 0.1 E 70 ° C,1,Reportar,,Calculo,9.74,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,None,"AIVEPET, S.R.L— REP. DOMINICANA\nReporte Fo...",ATLANTIC


In [250]:
df_obs_no_duplicates.loc[:,'file'] = df_obs_no_duplicates['file'].str.replace('.json', '.pdf')

In [251]:
df_obs_no_duplicates

,file,text,page
0,2023 08 030.pdf,Observación: Valores resaltados se encuentran ...,1
3,2023 08 030.pdf,Observación: Valores resaltados se encuentran ...,2
4,2023 08 030.pdf,Observación: Valores reportados se encuentran ...,3
5,2023 01 023.pdf,Observación: Valores reportados se encuentran ...,1
6,2023 01 023.pdf,Observación: Valores resaltados se encuentran ...,2
...,...,...,...
3036,2023 04 079.pdf,Observación: Valores resaltados se encuentran ...,3
3037,2023 05 079.pdf,Observación: Valores resaltados se encuentran ...,1
3038,2023 05 079.pdf,Observación: Valores resaltados se encuentran ...,2
3039,2023 05 079.pdf,Observación: Valores reportados se encuentran ...,3


In [253]:
df = df.merge(df_obs_no_duplicates,left_on=['ARCHIVO', 'NUM PAGINA'], right_on=['file', 'page'])

In [254]:
df.drop(['file', 'page'], axis = 1, inplace=True)

In [255]:
df.head()

,ANÁLISIS,UNIDADES,MIN,MAX,MÉTODO,RESULTADOS,NOMBRE ESTACIÓN,PRODUCTO,FECHA,NUM PAGINA,ARCHIVO,INCERTIDUMBRE/ UNCERTAINTY %,DIRECCIÓN,MARCA ESTACION,text
0,NUMERO DE OCTANO METODO RESEARCH (RON),-,95,-,ASTM D-2699,96.0,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,None,"AIVEPET, S.R.L— REP. DOMINICANA\nReporte Fo...",ATLANTIC,Observación: Valores resaltados se encuentran ...
1,NUMERO DE OCTANO METODO MOTOR (MON),.,82,-,ASTM D-2700,87.4,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,None,"AIVEPET, S.R.L— REP. DOMINICANA\nReporte Fo...",ATLANTIC,Observación: Valores resaltados se encuentran ...
2,CONTENIDO DE PLOMO,G/gal,1,0.02,ASTM D-3237,N/D,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,None,"AIVEPET, S.R.L— REP. DOMINICANA\nReporte Fo...",ATLANTIC,Observación: Valores resaltados se encuentran ...
3,PRESION A VAPOR REID (RVP) @ 100 º F,PSI,1,10.0,ASTM D-323,8.02,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,None,"AIVEPET, S.R.L— REP. DOMINICANA\nReporte Fo...",ATLANTIC,Observación: Valores resaltados se encuentran ...
4,RVP + 0.1 E 70 ° C,1,Reportar,,Calculo,9.74,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,None,"AIVEPET, S.R.L— REP. DOMINICANA\nReporte Fo...",ATLANTIC,Observación: Valores resaltados se encuentran ...


In [259]:
df = df.rename(columns={'text': 'OBSERVACIONES'})

In [261]:
df['CONFORMIDAD'] = None
df['CONFORMIDAD'] = df['OBSERVACIONES'].apply(lambda item: False if item.lower().find('fuera') > 0 else True)

In [265]:
df.head()

,ANÁLISIS,UNIDADES,MIN,MAX,MÉTODO,RESULTADOS,NOMBRE ESTACIÓN,PRODUCTO,FECHA,NUM PAGINA,ARCHIVO,INCERTIDUMBRE/ UNCERTAINTY %,DIRECCIÓN,MARCA ESTACION,OBSERVACIONES,CONFORMIDAD
0,NUMERO DE OCTANO METODO RESEARCH (RON),-,95,-,ASTM D-2699,96.0,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,None,"AIVEPET, S.R.L— REP. DOMINICANA\nReporte Fo...",ATLANTIC,Observación: Valores resaltados se encuentran ...,False
1,NUMERO DE OCTANO METODO MOTOR (MON),.,82,-,ASTM D-2700,87.4,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,None,"AIVEPET, S.R.L— REP. DOMINICANA\nReporte Fo...",ATLANTIC,Observación: Valores resaltados se encuentran ...,False
2,CONTENIDO DE PLOMO,G/gal,1,0.02,ASTM D-3237,N/D,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,None,"AIVEPET, S.R.L— REP. DOMINICANA\nReporte Fo...",ATLANTIC,Observación: Valores resaltados se encuentran ...,False
3,PRESION A VAPOR REID (RVP) @ 100 º F,PSI,1,10.0,ASTM D-323,8.02,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,None,"AIVEPET, S.R.L— REP. DOMINICANA\nReporte Fo...",ATLANTIC,Observación: Valores resaltados se encuentran ...,False
4,RVP + 0.1 E 70 ° C,1,Reportar,,Calculo,9.74,ATLANTIC BONAO\n(ANIANA),GASOLINA PREMIUM,2023-04-07 15:12:00,1,2023 07 014.pdf,None,"AIVEPET, S.R.L— REP. DOMINICANA\nReporte Fo...",ATLANTIC,Observación: Valores resaltados se encuentran ...,False


In [266]:
df.to_csv('./gas_station_dataset.csv')
df.to_parquet('./gas_station_dataset.parquet')